In [ ]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.document import DocumentTranslationClient, TranslatorCredential

# Replace with your Azure credentials and endpoint
text_analytics_key = "8ae68db6742b4ee9a8447bb388fb3f27"
text_analytics_endpoint = "https://project2revature-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/bfe46c0a-7f3f-4e3f-9b9b-f4fa44dc7ecd/classify/iterations/Iteration1/image"


translator_key = "c721a4fe4cc2457583f177564fbf697e"
translator_endpoint =  "https://openrevaturep2.openai.azure.com/"


# Initialize the Text Analytics client
text_analytics_client = TextAnalyticsClient(
    endpoint=text_analytics_endpoint, 
    credential=AzureKeyCredential(text_analytics_key)
)

# Initialize the Translator client
translator_client = DocumentTranslationClient(
    endpoint=translator_endpoint,
    credential=TranslatorCredential(translator_key)
)


Data Preprocessing and Translation

In [ ]:
def translate_text(text, target_language='en'):
    response = translator_client.translate(
        content=text, 
        target_language=target_language
    )
    return response.translations[0].text

# Example of translation
review = "Este es un buen producto"
translated_review = translate_text(review)
print(translated_review)  # Outputs: "This is a good product"


Sentiment Analysis

In [ ]:
def analyze_sentiment(reviews):
    response = text_analytics_client.analyze_sentiment(documents=reviews)
    sentiment_results = []
    for result in response:
        sentiment_results.append({
            "text": result.text,
            "sentiment": result.sentiment,
            "confidence_scores": result.confidence_scores
        })
    return sentiment_results

# Example of sentiment analysis
reviews = [translated_review]  # List of reviews
sentiments = analyze_sentiment(reviews)
print(sentiments)


Named Entity Recognition (NER) and Key Phrase Extraction

In [ ]:
def extract_entities_and_key_phrases(reviews):
    response = text_analytics_client.recognize_entities(documents=reviews)
    key_phrases_response = text_analytics_client.extract_key_phrases(documents=reviews)

    entity_results = []
    for idx, result in enumerate(response):
        entities = [entity.text for entity in result.entities]
        key_phrases = key_phrases_response[idx].key_phrases
        entity_results.append({
            "text": result.text,
            "entities": entities,
            "key_phrases": key_phrases
        })
    return entity_results

# Example of entity recognition and key phrase extraction
entities_key_phrases = extract_entities_and_key_phrases(reviews)
print(entities_key_phrases)


Data Storage

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('reviews_analysis.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE IF NOT EXISTS ReviewsAnalysis 
             (text TEXT, sentiment TEXT, entities TEXT, key_phrases TEXT)''')

# Insert data
for result in entities_key_phrases:
    c.execute("INSERT INTO ReviewsAnalysis (text, sentiment, entities, key_phrases) VALUES (?, ?, ?, ?)", 
              (result['text'], result['sentiment'], ', '.join(result['entities']), ', '.join(result['key_phrases'])))

# Commit and close
conn.commit()
conn.close()


Data Visualization

In [ ]:
import matplotlib.pyplot as plt

def plot_sentiment_distribution(sentiment_results):
    sentiments = [result["sentiment"] for result in sentiment_results]
    sentiment_counts = {sentiment: sentiments.count(sentiment) for sentiment in set(sentiments)}
    
    plt.bar(sentiment_counts.keys(), sentiment_counts.values())
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.show()

plot_sentiment_distribution(sentiments)
